# Regresión Polinomial

Hasta ahora hemos utilizado modelos lineales para tareas de regresión y clasificación. Esto quiere decir que el resultado de nuestra predicción depende de una combinación lineal de los features. Sin embargo, hemos visto que es util preprocesar los features antes de alimentar un modelo. Un preprocesado muy util, que permite capturar relaciones no lineales entre los features, es el _preprocesado polinomico_. El resultado de componer este preprocesado con un modelo lineal de regresión es lo que se conoce como _regresión polinómica_, pero también puede ser utilizado para clasificación con un regresor logístico.

A lo largo de este notebook:
- Nos encontraremos con un conjunto de datos que posee características no lineales
- Entrenaremos un regresor logístico para clasificar
- Realizaremos un preprocesamiento polinómico de los datos, y veremos esto modifica nuestros datos.
- Entrenaremos un regresor logístico polinómico, y lo compararemos al regresor logístico lineal.
- Exploraremos distintos grados del polinomio, en búsqueda de encontrar _el grado óptimo_.




## Utils

A continuación le presentamos una función que podrá ser de utilidad para plotear regiones y fronteras de decisión. Si quiere puede analizar que hace, o simplemente usarla en sus soluciones.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

def plot_regions(classifier, x, t):
    """Plot results from classification."""
    plt.figure(figsize=(9, 7))

    xx, yy = np.meshgrid(np.linspace(x[:, 0].min()-1, x[:, 0].max()+1, 200),
                         np.linspace(x[:, 1].min()-1, x[:, 1].max()+1, 200))

    # evaluate decision function
    Z = classifier.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # colour regions
    plt.pcolormesh(xx, yy, Z<0, cmap=plt.cm.bwr, shading='auto', alpha=0.4)
    # decision boundary
    plt.contour(xx, yy, 1/(1 + np.exp(-Z)), [0.05, 0.5, 0.95], colors=['0.5', 'k', '0.5'], zorder=1)

    xc1 = x[t == np.unique(t.flatten()).max()]
    xc2 = x[t == np.unique(t.flatten()).min()]

    plt.plot(*xc1.T, 'ob', mfc='None', label='C1')
    plt.plot(*xc2.T, 'or', mfc='None', label='C2')

    # Remove ticks
    plt.xticks(())
    plt.yticks(())
    plt.axis('tight')

    return


## Dataset

La siguiente línea de código generará un dataset sintético a utilizar en este notebook:

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
X, y = make_moons(300, noise=0.25, random_state=42)

## Ejercicio 1

- Examine el dataset y grafíquelo
- Utilice la función `train_test_split` para separar un 20% del dataset como conjunto de test. Este servirá para evaluar la performance del modelo sobre datos no vistos anteriormente.
- Plotee conjuntos (train y test) para visualizar la separación.

In [ ]:
from sklearn.model_selection import train_test_split
# Tu turno...


## Ejercicio 2

- Entrene un regresor logístico a los datos de entrenamiento. 
- Grafique la frontera de decisión (tip: puede utilizar la función utilitaria provista al comienzo del notebook)
- Mida la exactitud sobre el conjunto de entrenamiento.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Tu turno...

## Ejercicio 3

Un polinomio de grado $M$ es una combinación lineal de monomios de grado $0$ hasta $M$. Es decir, si miramos un polinomio univariable de segundo grado, este es de la forma:
$$w_0 + w_1 x + w_2 x^2 $$
que no es mas que una combinacion lineal de los monomios $x^0$ ($=1$), $x^1$ ($=x$) y $x^2$.

El caso de dos variables $x_1$ y $x_2$ (como nuestro dataset), tambien hay que tener en cuenta los terminos cruzados:
$$w_{(0,0)} + w_{(1,0)} x_1 + w_{(0,1)} x_2 + w_{(2,0)} x_1^2 + w_{(1,1)} x_1 x_2 + w_{(0,2)} x_2^2$$

es decir una combinacion lineal de los monomios $1$, $x_1$, $x_2$, $x_1^2$, $x_2^2$ y $x_1 x_2$.

Esto implica que un modelo polinomico no es mas que un modelo lineal, con un preprocesado que consiste en tomar los features del dataset, y devolver todos los monomios de estos hasta cierto grado. 

Este ya esta implementado en sklearn:



- Importe de `sklearn.preprocessing` el transformer `PolynomialFeatures`. Analice su firma. (_Pista: `?`_)
- Instancielo con el grado polinomico 1, 2 y 3. Para cada uno de ellos, transforme los datos (_Pista: `.fit_transform()`_), y examine como cambia el dataset, en particular mire el `shape`.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# Tu turno...


## Ejercicio 4

- Entrene un regresor logístico polinómico de grado 3 sobre el dataset de entrenamiento.
- Evalúe su exactitud sobre el conjuntos de entrenamiento. Compare con el obtenido en el ejercicio 2.

Para plotear la frontera de decisión, proponemos que sigas los siguientes pasos:
- Arme un pipeline de sklearn usando como primer paso el preprocesador polinomico, y como ultimo el regresor lineal. (_Pista: tanto `Pipeline` como `make_pipeline` de `sklearn.pipeline` pueden servir_)
- Entrene este pipeline sobre el conjunto de entrenamiento
- Pase el pipeline entrenado al argumento `classifier` de la función de `plot_regions` provista al comienzo del notebook.

_Opcional:_
- Examine como cambia el resultado incluyendo un `StandardScaler` entre el preprocesado polinomico y el regresor logístico. ¿Que cree que ocurre en este paso?

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler

# Tu turno

## Ejercicio 5

- Repita el ejercicio anterior para varios valores del grado polinómico (entre 1 y 10)
- Compare cualitativamente como cambian las fronteras de decisión. 
- Compare que pasa con la exactitud sobre el conjunto de entrenamiento.
- ¿Que cree que pasará con estos modelos cuando sean evaluados sobre datos que no han visto anteriormente?
- Compare que pasa con la exactitud sobre el conjunto de **evaluación**.
- ¿Que conclusiones extrae de esto?

-_Opcional_: Registre los valores de exactitud en ambos conjuntos (train y test), y ploteelas en función del grado polinómico usado. ¿Que observa?

In [ ]:
# Tu turno


## Ejercicio 6

- En función a lo observado, defina un método para elegir el _*grado polinómico óptimo*_. 
- Encuentre, segun este criterio, cual es el grado óptimo para este conjunto de datos.

In [ ]:
# Tu turno


## Formulario de asistencia

Por favor, no olviden completar el siguiente formulario antes del miércoles 09/04 a la 23:59.

[link a definir]